Import modules

In [1]:
import pandas as pd
import json
from pandas import json_normalize
import os

Read data from JSONs

In [5]:
PATH = '../../tempCorpus'
count = 0
df_list = []
for file in os.listdir(PATH):
        if file.endswith(".json"):
                if count % 10000 == 0 and count > 0:
                        print(f'{count} files processed')
                with open(rf'{PATH}/{file}') as f:
                        data = json.load(f)
                df_list.append(pd.DataFrame.from_dict(data, orient = "index"))
                count += 1

1000 files processed
2000 files processed
3000 files processed
4000 files processed
5000 files processed
6000 files processed
7000 files processed
8000 files processed
9000 files processed
10000 files processed
11000 files processed
12000 files processed
13000 files processed
14000 files processed
15000 files processed
16000 files processed
17000 files processed
18000 files processed
19000 files processed
20000 files processed
21000 files processed
22000 files processed
23000 files processed
24000 files processed
25000 files processed
26000 files processed
27000 files processed
28000 files processed
29000 files processed
30000 files processed
31000 files processed
32000 files processed
33000 files processed
34000 files processed
35000 files processed
36000 files processed
37000 files processed
38000 files processed
39000 files processed
40000 files processed
41000 files processed
42000 files processed
43000 files processed
44000 files processed
45000 files processed
46000 files process

Create Dataframe

In [6]:
dataframe = pd.concat(df_list, axis=1, ignore_index=True)